<a href="https://colab.research.google.com/github/vartikagpt10/EEG-based-ERP-Detection-for-practical-BCI/blob/main/CNN%2Bweighted_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Extract Data

In [ ]:
!pip install --upgrade gdown pyriemann
!gdown https://drive.google.com/uc?id=1RCZlEI1P0NpCqNkjlQMPnTAOU2phFJkq -O /tmp/Data.zip

import zipfile
local_zip = '//tmp/Data.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp/Data')
zip_ref.close()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp36-none-any.whl size=9694 sha256=b58eeb203971d76b2a3f21d0de965d6cad876b1679e5655992d43f1dd61cdf2a
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
Downloading...
From: https://drive.google.com/uc?id=1RCZlEI1P0NpCqNkjlQMPnTAOU2phFJkq
To: /tmp/Data.zip
226MB [00:01, 203MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.signal import filtfilt, cheby2
import scipy

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
train_folder_path = '/tmp/Data/Training set/'
validation_folder_path = '/tmp/Data/Validation set/'

# Data Extraction

In [ ]:
c=0
for i in os.listdir(train_folder_path):
  c+=1
  if c==1:
    temp=sio.loadmat(train_folder_path+i)
    dat=np.squeeze(temp['epo_tr']['x']).tolist()
    dat=np.array(dat)
    lab=np.squeeze(temp['epo_tr']['y']).tolist()
    lab=np.array(lab)[0,:]
    training_data=dat
    training_labels=lab
    continue
  temp=sio.loadmat(train_folder_path+i)
  dat=np.squeeze(temp['epo_tr']['x']).tolist()
  dat=np.array(dat)
  lab=np.squeeze(temp['epo_tr']['y']).tolist()
  lab=np.array(lab)[0,:]
  training_data=np.concatenate((training_data,dat),axis=2)
  training_labels=np.concatenate((training_labels,lab))
training_data=training_data.reshape((2700,56,100))
training_data=training_data[:,:32,:]
print("Training data shape:",training_data.shape)
print("Training labels shape:",training_labels.shape)

Training data shape: (2700, 32, 100)
Training labels shape: (2700,)


In [ ]:
c=0
for i in os.listdir(validation_folder_path):
  c+=1
  if c==1:
    temp=sio.loadmat(validation_folder_path+i)
    dat=np.squeeze(temp['epo_val']['x']).tolist()
    dat=np.array(dat)
    lab=np.squeeze(temp['epo_val']['y']).tolist()
    lab=np.array(lab)[0,:]
    validation_data=dat
    validation_labels=lab
    continue
  temp=sio.loadmat(validation_folder_path+i)
  dat=np.squeeze(temp['epo_val']['x']).tolist()
  dat=np.array(dat)
  lab=np.squeeze(temp['epo_val']['y']).tolist()
  lab=np.array(lab)[0,:]
  validation_data=np.concatenate((validation_data,dat),axis=2)
  validation_labels=np.concatenate((validation_labels,lab))
validation_data=validation_data.reshape((900,56,100))
validation_data=validation_data[:,:32,:]
print("Validation data shape:",validation_data.shape)
print("Validation labels shape:",validation_labels.shape)

Validation data shape: (900, 32, 100)
Validation labels shape: (900,)


In [ ]:
# dat=np.squeeze(temp['epo_val']['y']).tolist()
# dat=np.array(dat)[0,:]
# dat

# Filtering

In [ ]:
def chebyBandpass(signal, lowcut, fs, order):
  nyq=0.5*fs
  low = lowcut/nyq
  # high = highcut / nyq
  b, a = cheby2(order,12, low, btype='low')
  filtered = filtfilt(b, a, signal)
  return filtered

def filterall(data):
  filtered=[]
  for trial in data:
    temp_filtered=[]
    for channel in trial:
      temp_filtered.append(chebyBandpass(channel,20,100,3))
    filtered.append(temp_filtered)
  return np.array(filtered)

filt_train=filterall(training_data)
filt_val=filterall(validation_data)

# Classification

##CNN Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, Activation, AveragePooling2D, Dropout, SeparableConv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical

###EEGNet Model

In [ ]:
def get_model(flt_size=32, drop=0.25):
  #build model
  model = tf.keras.Sequential()
  model.add(Conv2D(64, (2, flt_size), padding="same", input_shape=(32,100,1)))
  model.add(BatchNormalization())
  C = 32
  model.add(DepthwiseConv2D((C, 1), padding="valid", depth_multiplier=2))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(AveragePooling2D(pool_size=(1, 4), padding="valid"))
  model.add(Dropout(drop))
  model.add(SeparableConv2D(16, (1, 16), padding="same"))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(AveragePooling2D(pool_size=(1, 8), padding="valid"))
  model.add(Dropout(drop))
  model.add(Flatten())
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

  return model

In [ ]:
filt_train = filt_train[:180,:,:]
training_labels = training_labels[:180]
filt_val = filt_val[:180,:,:]
validation_labels = validation_labels[:180]
training_labels.shape

(180, 1)

In [ ]:
from sklearn.utils import class_weight

# Calculate the weights for each class so that we can balance the data
weights = class_weight.compute_class_weight('balanced',
                                            np.unique(training_labels),
                                            training_labels)

print(weights)

In [ ]:
weights = {0: weights[0], 1: weights[1]}
weights

{0: 0.5960264900662252, 1: 3.103448275862069}

In [ ]:
filt_train.shape

(180, 32, 100)

In [ ]:
filt_train = filt_train[..., np.newaxis]
training_labels = training_labels[...,np.newaxis]
filt_val = filt_val[..., np.newaxis]
validation_labels = validation_labels[..., np.newaxis]

In [ ]:
model_new = get_model()
opt = optimizers.Adam(learning_rate=0.02)
model_new.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model_new.fit(filt_train, training_labels, epochs=200, class_weight=weights)

Epoch 1/200
6/6 [==============================] - 0s 8ms/step - loss: 0.8233 - accuracy: 0.4500
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6818 - accuracy: 0.6167
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6714 - accuracy: 0.5056
Epoch 4/200
6/6 [==============================] - 0s 7ms/step - loss: 0.6943 - accuracy: 0.3889
Epoch 5/200
6/6 [==============================] - 0s 7ms/step - loss: 0.6462 - accuracy: 0.6056
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6501 - accuracy: 0.6833
Epoch 7/200
6/6 [==============================] - 0s 7ms/step - loss: 0.6432 - accuracy: 0.6833
Epoch 8/200
6/6 [==============================] - 0s 6ms/step - loss: 0.6394 - accuracy: 0.6833
Epoch 9/200
6/6 [==============================] - 0s 6ms/step - loss: 0.6432 - accuracy: 0.4722
Epoch 10/200
6/6 [==============================] - 0s 6ms/step - loss: 0.6477 - accuracy: 0.5389
Epoch 11/200
6/6 [===========

In [ ]:
predictions = model_new.predict_proba(filt_val)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
temp=scipy.io.loadmat('/content/drive/My Drive/Test set/Data_Sample01.mat')

In [ ]:
dat=np.squeeze(temp['epo_te']['x']).tolist()
dat=np.array(dat)

test_data=dat
#test_data=np.concatenate((test_data,dat),axis=2)
test_data = test_data[:, :32,:]

In [ ]:
test_data = test_data.T

In [ ]:
test_data = test_data[..., np.newaxis]

In [ ]:
test_data.shape

(60, 32, 100, 1)

In [ ]:
predsubj1 = model_new.predict_proba(test_data)

In [ ]:
predsubj1[:60,0]

array([0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666,
       0.46799666, 0.46799666, 0.46799666, 0.46799666, 0.46799666],
      dtype=float32)

In [ ]:
predsubj1.shape

(60, 1)